<a href="https://colab.research.google.com/github/xlomin8/Final_project_Speech_synthesis_sm/blob/main/infer_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **음성 합성**

## **1. 구글 드라이브 마운트**
---
마운트할 구글 드라이브 내에 다음 파일들이 존재하는지 확인

- `/Colab Notebooks/data/glowtts-v2/model_file.pth.tar`
- `/Colab Notebooks/data/glowtts-v2/config.json`
- `/Colab Notebooks/data/hifigan-v2/model_file.pth.tar`
- `/Colab Notebooks/data/hifigan-v2/config.json`
---
존재하지 않는다면, [glowtts-v2.zip](https://drive.google.com/file/d/1DMKLdfZ_gzc_z0qDod6_G8fEXj0zCHvC/view?usp=sharing), [hifigan-v2.zip](https://drive.google.com/file/d/1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU/view?usp=sharing)을 내려받아 준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **2. 필수 라이브러리 및 함수 불러오기**

In [ ]:
import os
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
!git clone --depth 1 https://github.com/sce-tts/g2pK.git
%cd /content/TTS
!pip install -q --no-cache-dir -e .
%cd /content/g2pK
!pip install -q --no-cache-dir "konlpy" "jamo" "nltk" "python-mecab-ko"
!pip install -q --no-cache-dir -e .

/content
Cloning into 'TTS'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (413/413), done.
remote: Total 447 (delta 56), reused 222 (delta 22), pack-reused 0
Receiving objects: 100% (447/447), 13.77 MiB | 26.90 MiB/s, done.
Resolving deltas: 100% (56/56), done.
Cloning into 'g2pK'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 0), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.
/content/TTS
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 69 kB 5.9 MB/s 
     |████████████████████████████████| 20.1 MB 11.1 MB/s 
     |████████████████████████████████| 183 kB 80.7 MB/s 
     |████████████████████████████████| 3.2 MB 48.0 MB/s 
     |████████████████████████

In [ ]:
%cd /content/g2pK
import g2pk
g2p = g2pk.G2p()

/content/g2pK


[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
%cd /content/TTS
import re
import sys
from unicodedata import normalize
import IPython

from TTS.utils.synthesizer import Synthesizer

def normalize_text(text):
    text = text.strip()

    for c in ",;:":
        text = text.replace(c, ".")
    text = remove_duplicated_punctuations(text)

    text = jamo_text(text)

    text = g2p.idioms(text)
    text = g2pk.english.convert_eng(text, g2p.cmu)
    text = g2pk.utils.annotate(text, g2p.mecab)
    text = g2pk.numerals.convert_num(text)
    text = re.sub("/[PJEB]", "", text)

    text = alphabet_text(text)

    # remove unreadable characters
    text = normalize("NFD", text)
    text = "".join(c for c in text if c in symbols)
    text = normalize("NFC", text)

    text = text.strip()
    if len(text) == 0:
        return ""

    # only single punctuation
    if text in '.!?':
        return punctuation_text(text)

    # append punctuation if there is no punctuation at the end of the text
    if text[-1] not in '.!?':
        text += '.'

    return text


def remove_duplicated_punctuations(text):
    text = re.sub(r"[.?!]+\?", "?", text)
    text = re.sub(r"[.?!]+!", "!", text)
    text = re.sub(r"[.?!]+\.", ".", text)
    return text


def split_text(text):
    text = remove_duplicated_punctuations(text)

    texts = []
    for subtext in re.findall(r'[^.!?\n]*[.!?\n]', text):
        texts.append(subtext.strip())

    return texts


def alphabet_text(text):
    text = re.sub(r"(a|A)", "에이", text)
    text = re.sub(r"(b|B)", "비", text)
    text = re.sub(r"(c|C)", "씨", text)
    text = re.sub(r"(d|D)", "디", text)
    text = re.sub(r"(e|E)", "이", text)
    text = re.sub(r"(f|F)", "에프", text)
    text = re.sub(r"(g|G)", "쥐", text)
    text = re.sub(r"(h|H)", "에이치", text)
    text = re.sub(r"(i|I)", "아이", text)
    text = re.sub(r"(j|J)", "제이", text)
    text = re.sub(r"(k|K)", "케이", text)
    text = re.sub(r"(l|L)", "엘", text)
    text = re.sub(r"(m|M)", "엠", text)
    text = re.sub(r"(n|N)", "엔", text)
    text = re.sub(r"(o|O)", "오", text)
    text = re.sub(r"(p|P)", "피", text)
    text = re.sub(r"(q|Q)", "큐", text)
    text = re.sub(r"(r|R)", "알", text)
    text = re.sub(r"(s|S)", "에스", text)
    text = re.sub(r"(t|T)", "티", text)
    text = re.sub(r"(u|U)", "유", text)
    text = re.sub(r"(v|V)", "브이", text)
    text = re.sub(r"(w|W)", "더블유", text)
    text = re.sub(r"(x|X)", "엑스", text)
    text = re.sub(r"(y|Y)", "와이", text)
    text = re.sub(r"(z|Z)", "지", text)

    return text


def punctuation_text(text):
    # 문장부호
    text = re.sub(r"!", "느낌표", text)
    text = re.sub(r"\?", "물음표", text)
    text = re.sub(r"\.", "마침표", text)

    return text


def jamo_text(text):
    # 기본 자모음
    text = re.sub(r"ㄱ", "기역", text)
    text = re.sub(r"ㄴ", "니은", text)
    text = re.sub(r"ㄷ", "디귿", text)
    text = re.sub(r"ㄹ", "리을", text)
    text = re.sub(r"ㅁ", "미음", text)
    text = re.sub(r"ㅂ", "비읍", text)
    text = re.sub(r"ㅅ", "시옷", text)
    text = re.sub(r"ㅇ", "이응", text)
    text = re.sub(r"ㅈ", "지읒", text)
    text = re.sub(r"ㅊ", "치읓", text)
    text = re.sub(r"ㅋ", "키읔", text)
    text = re.sub(r"ㅌ", "티읕", text)
    text = re.sub(r"ㅍ", "피읖", text)
    text = re.sub(r"ㅎ", "히읗", text)
    text = re.sub(r"ㄲ", "쌍기역", text)
    text = re.sub(r"ㄸ", "쌍디귿", text)
    text = re.sub(r"ㅃ", "쌍비읍", text)
    text = re.sub(r"ㅆ", "쌍시옷", text)
    text = re.sub(r"ㅉ", "쌍지읒", text)
    text = re.sub(r"ㄳ", "기역시옷", text)
    text = re.sub(r"ㄵ", "니은지읒", text)
    text = re.sub(r"ㄶ", "니은히읗", text)
    text = re.sub(r"ㄺ", "리을기역", text)
    text = re.sub(r"ㄻ", "리을미음", text)
    text = re.sub(r"ㄼ", "리을비읍", text)
    text = re.sub(r"ㄽ", "리을시옷", text)
    text = re.sub(r"ㄾ", "리을티읕", text)
    text = re.sub(r"ㄿ", "리을피읍", text)
    text = re.sub(r"ㅀ", "리을히읗", text)
    text = re.sub(r"ㅄ", "비읍시옷", text)
    text = re.sub(r"ㅏ", "아", text)
    text = re.sub(r"ㅑ", "야", text)
    text = re.sub(r"ㅓ", "어", text)
    text = re.sub(r"ㅕ", "여", text)
    text = re.sub(r"ㅗ", "오", text)
    text = re.sub(r"ㅛ", "요", text)
    text = re.sub(r"ㅜ", "우", text)
    text = re.sub(r"ㅠ", "유", text)
    text = re.sub(r"ㅡ", "으", text)
    text = re.sub(r"ㅣ", "이", text)
    text = re.sub(r"ㅐ", "애", text)
    text = re.sub(r"ㅒ", "얘", text)
    text = re.sub(r"ㅔ", "에", text)
    text = re.sub(r"ㅖ", "예", text)
    text = re.sub(r"ㅘ", "와", text)
    text = re.sub(r"ㅙ", "왜", text)
    text = re.sub(r"ㅚ", "외", text)
    text = re.sub(r"ㅝ", "워", text)
    text = re.sub(r"ㅞ", "웨", text)
    text = re.sub(r"ㅟ", "위", text)
    text = re.sub(r"ㅢ", "의", text)

    return text


def normalize_multiline_text(long_text):
    texts = split_text(long_text)
    normalized_texts = [normalize_text(text).strip() for text in texts]
    return [text for text in normalized_texts if len(text) > 0]

def synthesize(text):
    wavs = synthesizer.tts(text, None, None)
    return wavs

/content/TTS


/usr/local/lib/python3.7/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


## **3. 학습한 모델 불러오기**

In [ ]:
synthesizer = Synthesizer(
    "/content/drive/MyDrive/Colab Notebooks/data2/glowtts-v2/glowtts-v2-August-08-2022_08+27AM-3aa165a/best_model.pth.tar",
    "/content/drive/MyDrive/Colab Notebooks/data2/glowtts-v2/glowtts-v2-August-08-2022_08+27AM-3aa165a/config.json",
    None,
    "/content/drive/MyDrive/Colab Notebooks/data2/hifigan-v2/hifigan-v2-August-09-2022_02+38AM-3aa165a/best_model.pth.tar",
    "/content/drive/MyDrive/Colab Notebooks/data2/hifigan-v2/hifigan-v2-August-09-2022_02+38AM-3aa165a/config.json",
    None,
    None,
    False,
)

 > Using model: glow_tts


/content/TTS/TTS/tts/layers/glow_tts/glow.py:84: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2491.)
  w_init = torch.qr(torch.FloatTensor(self.num_splits, self.num_splits).normal_())[0]


 > Generator Model: hifigan_generator
Removing weight norm...


In [ ]:
symbols = synthesizer.tts_config.characters.characters

## **4. synthesizer 모델 저장**

In [ ]:
%cd /content/TTS
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data2/sythesizer"

/content/TTS


In [ ]:
%cd /content/TTS
!(python TTS/utils/synthesizer.py \
#     --config_path "/content/drive/My Drive/Colab Notebooks/data2/glowtts-v2/config.json" \
#     --coqpit.datasets.0.path "/content/TTS/filelists"  \
#     --coqpit.audio.stats_path "/content/drive/My Drive/Colab Notebooks/data2/glowtts-v2/scale_stats_new.npy"  \
#     --coqpit.test_sentences_file "/content/TTS/test_sentences.txt"  \
#     --coqpit.output_path "/content/drive/My Drive/Colab Notebooks/data2/glowtts-v2/"  \
#     --coqpit.num_loader_workers 2  \
#     --coqpit.num_val_loader_workers 2  \
#     --restore_path "/content/drive/My Drive/Colab Notebooks/data2/glowtts-v2/model_file.pth.tar")

/content/TTS
/bin/bash: -c: line 3: syntax error: unexpected end of file


In [ ]:
synthesizer.save("/content/drive/My Drive/Colab Notebooks/data2/sythesizer/model.pth")

AttributeError: ignored

In [ ]:
# synthesizer = Synthesizer(
#     "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/model_file.pth.tar",
#     "/content/drive/My Drive/Colab Notebooks/data/glowtts-v2/config.json",
#     None,
#     "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/model_file.pth.tar",
#     "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json",
#     None,
#     None,
#     False,
# )
# symbols = synthesizer.tts_config.characters.characters

 > Using model: glow_tts
 > Generator Model: hifigan_generator
Removing weight norm...


## **5. 음성 합성**
---
`long_text`의 값을 변경하여 다른 문장의 합성 가능

In [ ]:
texts = """
아래 문장들은 모델 학습을 위해 사용하지 않은 문장들입니다.
서울특별시 특허허가과 허가과장 허과장.
경찰청 철창살은 외철창살이고 검찰청 철창살은 쌍철창살이다.
지향을 지양으로 오기하는 일을 지양하는 언어 습관을 지향해야 한다.
그러니까 외계인이 우리 생각을 읽고 우리 생각을 우리가 다시 생각토록 해서 그 생각이 마치 우리가 생각한 것인 것처럼 속였다는 거냐?
안 촉촉한 초코칩 나라에 살던 안 촉촉한 초코칩이 촉촉한 초코칩 나라의 촉촉한 초코칩을 보고 촉촉한 초코칩이 되고 싶어서 촉촉한 초코칩 나라에 갔는데 촉촉한 초코칩 나라의 촉촉한 문지기가 넌 촉촉한 초코칩이 아니고 안 촉촉한 초코칩이니까 안 촉촉한 초코칩 나라에서 살라고 해서 안 촉촉한 초코칩은 촉촉한 초코칩이 되는 것을 포기하고 안 촉촉한 눈물을 흘리며 안 촉촉한 초코칩 나라로 돌아갔다.
"""
# 깊은 바닷속 인어와 온갖 물고기들이 트리톤 왕의 왕궁으로 모여들었어요. 트리톤 왕의 막내딸인 인어공주 에리얼이 처음으로 음악회에서 노래하는 걸 보기 위해서였어요.

for text in normalize_multiline_text(texts):
    wav = synthesizer.tts(text, None, None)
    IPython.display.display(IPython.display.Audio(wav, rate=22050))  

 > Text splitted to sentences.
['아래 문장들은 모델 학습을 위해 사용하지 않은 문장들입니다.']
 > Processing time: 1.4832985401153564
 > Real-time factor: 0.2678069960168316


 > Text splitted to sentences.
['서울특별시 특허허가과 허가과장 허과장.']
 > Processing time: 1.0201606750488281
 > Real-time factor: 0.23680460338582893


 > Text splitted to sentences.
['경찰청 철창쌀은 외철창쌀이고 검찰청 철창쌀은 쌍철창쌀이다.']
 > Processing time: 1.5115840435028076
 > Real-time factor: 0.21168090234247605


 > Text splitted to sentences.
['지향을 지양으로 오기하는 일을 지양하는 언어 습관을 지향해야 한다.']
 > Processing time: 1.3472018241882324
 > Real-time factor: 0.21057190812741383


 > Text splitted to sentences.
['그러니까 외계인이 우리 생각을 읽고 우리 생각을 우리가 다시 생각토록 해서 그 생각이 마치 우리가 생각한 것인 것처럼 속였다는 거냐?']
 > Processing time: 2.285602569580078
 > Real-time factor: 0.20830937379819753


 > Text splitted to sentences.
['안 촉촉한 초코칩 나라에 살던 안 촉촉한 초코칩이 촉촉한 초코칩 나라의 촉촉한 초코칩을 보고 촉촉한 초코칩이 되고 싶어서 촉촉한 초코칩 나라에 갔는데 촉촉한 초코칩 나라의 촉촉한 문지기가 넌 촉촉한 초코칩이 아니고 안 촉촉한 초코칩이니까 안 촉촉한 초코칩 나라에서 살라고 해서 안 촉촉한 초코칩은 촉촉한 초코칩이 되는 것을 포기하고 안 촉촉한 눈물을 흘리며 안 촉촉한 초코칩 나라로 돌아갔다.']
 > Processing time: 6.425378799438477
 > Real-time factor: 0.21195813864030807
